In [17]:
import bs4
import pickle
import requests
import csv

In [18]:
# DATA SCRAPING FROM "https://www.bbcgoodfood.com/" WEBSITE
req_page = requests.get("https://www.bbcgoodfood.com/")
html_file = bs4.BeautifulSoup(req_page.text, "html.parser")

In [29]:
guide_links = []
tags = ['/recipes/']
for link in html_file.find_all('a', role='menuitem'):
  try:
    for tag in tags:
      if tag in link['href']:
        guide_links.append(link['href'])
      else:
        continue
  except:
    continue

print("Successfully Found Guide Links!!")
print(len(guide_links))

Successfully Found Guide Links!!
44


In [49]:
recipe_names = []
ingredients_list = []
instructions_list = []
image_links = []
cook_time=[]

def scrape_recipe(recipe_url):
    recipe_response = requests.get(recipe_url)
    recipe_soup = bs4.BeautifulSoup(recipe_response.text, 'html.parser')

    recipe_name = recipe_soup.find('h1', class_="heading-1").text.strip()
    recipe_names.append(recipe_name)

    try:
        prep_time = recipe_soup.find('time').text.strip()
        cook_time.append(prep_time)
    except:
        cook_time.append(None)

    ingredients = recipe_soup.find_all('li', class_='pb-xxs pt-xxs list-item list-item--separator')
    ingredients_text = [ingredient.text.strip() for ingredient in ingredients]
    ingredients_list.append(ingredients_text)

    instructions = recipe_soup.find_all('li', class_='pb-xs pt-xs list-item')
    instructions_text = [instruction.text.strip() for instruction in instructions]
    instructions_list.append(instructions_text)

    image = recipe_soup.find('img', class_="image__img")['src']
    image_links.append(image)

#--------------------------------------------------------------------------------------------

def get_recipe_links(main_page_url):
    recipe_links = []
    for guide_link in guide_links:
        response = requests.get(main_page_url + guide_link)
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a', {'class' : 'link d-block', 'data-component' : 'Link'}):
            if '/recipes/' in link['href']:
                recipe_links.append("https://www.bbcgoodfood.com" + link['href'])
              
    return recipe_links


main_page_url = 'https://www.bbcgoodfood.com'
recipe_links = get_recipe_links(main_page_url)

for recipe_link in recipe_links:
    scrape_recipe(recipe_link)


with open('bbc_good_food_recipes.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Recipe Name', 'Ingredients', 'Instructions', 'Image Link', 'Cook Time'])
    for i in range(len(recipe_names)):
        if cook_time[i] is not None:
            writer.writerow([recipe_names[i], ", ".join(ingredients_list[i]), " ".join(instructions_list[i]), image_links[i], cook_time[i]])
        else:
            continue

print("Scraping complete! Data saved to bbc_good_food_recipes.csv")

Scraping complete! Data saved to bbc_good_food_recipes.csv


In [50]:
import pandas as pd
df = pd.read_csv("C:/Users/Akanksha/PROJECTS/Web_Scraping_Projects/CookMatch/bbc_good_food_recipes.csv")
print(df.shape)
df.head()

(697, 5)


,Recipe Name,Ingredients,Instructions,Image Link,Cook Time
0,Mango chutney & chicken sliders,"100ml natural yogurt, 3 tbsp mayonnaise, ½ sma...","STEP 1Mix the yogurt, mayonnaise and mint toge...",https://images.immediate.co.uk/production/vola...,20 mins
1,Peach panzanella,"6 thin slices of brioche loaf, cut into cubes,...","STEP 1Make the dressing by whisking the oil, l...",https://images.immediate.co.uk/production/vola...,20 mins
2,Crispy shredded chicken,"320g pack chicken mini breast fillets, 2½ tbs...",STEP 1Slice the chicken into thin strips. Pour...,https://images.immediate.co.uk/production/vola...,10 mins
3,Pulled pork sloppy joe dogs,"1 tbsp sunflower oil, 2 onions, halved and sli...",STEP 1Heat the oil in a pan and fry the onions...,https://images.immediate.co.uk/production/vola...,20 mins
4,Panuozzo sandwich,"300g strong white bread flour, plus extra for ...","STEP 1Put the flour in a large bowl, then stir...",https://images.immediate.co.uk/production/vola...,20 mins


In [51]:
from skimpy import skim
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 697    │ │ string      │ 5     │                                                          │
│ │ Number of columns │ 5      │ └─────────────┴───────┘                                                          │
│ └───────────────────┴────────┘                                                                                  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                ┃ NA     ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Recipe Name                │      0 │          0 │                          4.1 │                     2852 │  │
│ │ Ingredients                │      0 │          0 │                           54 │                    37402 │  │
│ │ Instructions               │      0 │          0 │                          160 │                   109372 │  │
│ │ Image Link                 │      0 │          0 │                            1 │                      697 │  │
│ │ Cook Time                  │      0 │          0 │                            2 │                     1409 │  │
│ └────────────────────────────┴────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯